## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
MODULE_BASE_PATH = '/kaggle/input/bert-base-uncased-huggingface/bert-base-uncased'
MODULE_PATH = MODULE_BASE_PATH + '/bert-base-uncased-tf_model.h5'
VOCAB_PATH = MODULE_BASE_PATH + '/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/' + '*last_epoch.h5')
model_path_list += glob.glob('/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/model_fold_1_last_epoch.h5
/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/model_fold_2_last_epoch.h5
/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/model_fold_3_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_1_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_2_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

In [4]:
### BERT auxiliar function
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length, ignore_first_sep=True):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if ignore_first_sep:
                ignore_first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input_2_tail(ft_input1, ft_input2, tokenizer, max_sequence_length, 
                  max_len_ft1=254, max_len_ft2=255):

    ft1 = tokenizer.tokenize(ft_input1)
    ft2 = tokenizer.tokenize(ft_input2)
    
    ft1_len = len(ft1)
    ft2_len = len(ft2)

    if (ft1_len + ft2_len + 3) > max_sequence_length:
      
        if max_len_ft2 > ft2_len:
            ft2_new_len = ft2_len 
            ft1_new_len = max_len_ft1 + (max_len_ft2 - ft2_len)
        elif max_len_ft1 > ft1_len:
            ft2_new_len = max_len_ft2 + (max_len_ft1 - ft1_len)
            ft1_new_len = ft1_len
        else:
            ft2_new_len = max_len_ft2
            ft1_new_len = max_len_ft1
                        
        if ft1_new_len + ft2_new_len + 3 > max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (ft1_new_len + ft2_new_len + 3)))
        
        ft1 = ft1[-ft1_new_len:]
        ft2 = ft2[-ft2_new_len:]
        
    return ft1, ft2

def _convert_to_bert_inputs_2(feature1, feature2, tokenizer, max_sequence_length, ignore_first_sep=True):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + feature1 + ["[SEP]"] + feature2 + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length, ignore_first_sep)

    return [input_ids, input_masks, input_segments]

def compute_input_arays_2_tail(df, columns, tokenizer, max_sequence_length, 
                          max_len_ft1, max_len_ft2, ignore_first_sep=True):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in df[columns].iterrows():
        ft1, ft2 = instance[columns[0]], instance[columns[1]]

        ft1, ft2 = _trim_input_2_tail(ft1, ft2, tokenizer, max_sequence_length, 
                                      max_len_ft1, max_len_ft2)

        ids, masks, segments = _convert_to_bert_inputs_2(ft1, ft2, tokenizer, max_sequence_length, 
                                                         ignore_first_sep)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

## Pre-process data

In [5]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# Model parameters

In [6]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [7]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features HEAD
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Test features TAIL
X_test_tail = compute_input_arays_2_tail(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                                         MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
              + compute_input_arays_2_tail(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                           MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [8]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_model = TFBertModel.from_pretrained(MODULE_PATH, config=bert_config)
    sequence_output_question, pooled_output_question = bert_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = bert_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [9]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += ((model.predict(X_test) + model.predict(X_test_tail)) / 2) / len(model_path_list)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test


n=test['url'].apply(lambda x:(('ell.stackexchange.com' in x) or ('english.stackexchange.com' in x))).tolist()
spelling=[]
for x in n:
    if x:
        spelling.append(0.5)
    else:
        spelling.append(0.)
        
submission['question_type_spelling']=spelling


submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.951851,0.624649,0.218848,0.593511,0.481652,0.513607,0.730492,0.663515,0.541617,...,0.921897,0.916928,0.603116,0.958320,0.977360,0.825148,0.016827,0.012673,0.855827,0.930494
1,46,0.899056,0.550803,0.006747,0.778666,0.738384,0.912770,0.556272,0.485422,0.082373,...,0.662630,0.935734,0.631244,0.970180,0.980916,0.880051,0.931029,0.128600,0.070630,0.912042
2,70,0.918654,0.742780,0.034487,0.787189,0.873841,0.924990,0.650298,0.489719,0.105165,...,0.911222,0.945910,0.636481,0.968667,0.976531,0.873292,0.025506,0.024602,0.881857,0.916548
3,132,0.863782,0.444818,0.008617,0.706047,0.727125,0.896989,0.534084,0.409940,0.103908,...,0.699427,0.940109,0.691700,0.955776,0.974127,0.898588,0.731780,0.136647,0.658305,0.905727
4,200,0.937407,0.480584,0.026983,0.874452,0.773063,0.919843,0.621225,0.613303,0.132750,...,0.621889,0.908113,0.652873,0.965058,0.969506,0.830223,0.276420,0.120432,0.535052,0.926987


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.908739,0.626782,0.037608,0.756722,0.792564,0.839850,0.598682,0.500672,0.221871,...,0.811767,0.932711,0.663252,0.962445,0.971327,0.867514,0.506175,0.125132,0.506872,0.920481
std,2812.670060,0.044055,0.154684,0.066889,0.100668,0.110231,0.126631,0.058915,0.092264,0.210215,...,0.096904,0.031660,0.051340,0.015400,0.016859,0.049319,0.329225,0.062210,0.297032,0.021703
min,39.000000,0.741641,0.297069,0.002633,0.230908,0.345963,0.205884,0.500768,0.337832,0.007281,...,0.541280,0.750314,0.503111,0.882064,0.839361,0.635344,0.005806,0.008609,0.021406,0.829187
25%,2572.000000,0.878547,0.491375,0.008061,0.704433,0.740416,0.815173,0.551520,0.433320,0.065671,...,0.731142,0.919724,0.629241,0.955867,0.966357,0.843614,0.166198,0.078653,0.250641,0.909200
50%,5093.000000,0.912447,0.611369,0.012425,0.759758,0.807836,0.883609,0.584913,0.476220,0.134279,...,0.812039,0.940953,0.661620,0.965678,0.976262,0.878246,0.574638,0.125428,0.504085,0.924069
75%,7482.000000,0.943722,0.766470,0.030422,0.817850,0.862170,0.917834,0.634065,0.550263,0.340088,...,0.907348,0.954343,0.691519,0.972567,0.981577,0.901875,0.811141,0.166237,0.787781,0.935630
max,9640.000000,0.988111,0.927874,0.531505,0.975780,0.984420,0.975111,0.767412,0.779885,0.939748,...,0.969835,0.980077,0.827015,0.986688,0.993042,0.953753,0.962662,0.315606,0.984051,0.960781
